In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

data = [
(1,"Manish",26,75000,"bihar","nominee1"),
(2,"Nikita",23,100000,"uttarpradesh","nominee2"),
(3,"Pritam",22,150000,"Bangalore","India","nominee3"),
(4,"Prantosh",17,200000,"Kolkata","India","nominee4"),
(5,"Vikash",31,300000,"","nominee5")
]

# id,name,age,salary,address,nominee
schema = StructType([
    StructField("id", IntegerType()),
    StructField("name", StringType()),
    StructField("age", IntegerType()),
    StructField("salary", IntegerType()),
    StructField("address", StringType()),
    StructField("nominee", StringType()),
])

df = spark.createDataFrame(data=data, schema=schema)

# ValueError: Length of object (7) does not match with length of fields (6)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-3408796042947017>:21
     11 # id,name,age,salary,address,nominee
     12 schema = StructType([
     13     StructField("id", IntegerType()),
     14     StructField("name", StringType()),
   (...)
     18     StructField("nominee", StringType()),
     19 ])
---> 21 df = spark.createDataFrame(data=data, schema=schema)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1216, in SparkSession.createDataFrame(self, data, schema, samplingRatio, verifySchema)


In [0]:
%fs

ls /FileStore/tables/employee_file.txt


path,name,size,modificationTime
dbfs:/FileStore/tables/employee_file.txt,employee_file.txt,225,1743943942000


In [0]:
# PERMISSIVE - 5 rec

employee_permissive_df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .option("mode", "PERMISSIVE")\
    .load("dbfs:/FileStore/tables/employee_file.txt")


employee_permissive_df.show()
"""
PERMISSIVE doesn't validate if extra columns are present
Shows all rows from csv
+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2| 
|  3|  Pritam| 22|150000|   Bangalore|   India| <--- Wrong rec ignored nominee
|  4|Prantosh| 17|200000|     Kolkata|   India| <--- Wrong rec ignored nominee
|  5|  Vikash| 31|300000|        null|nominee5| <--- null as value is not present
+---+--------+---+------+------------+--------+
"""

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|   India|
|  4|Prantosh| 17|200000|     Kolkata|   India|
|  5|  Vikash| 31|300000|        null|nominee5|
+---+--------+---+------+------------+--------+

Out[8]: '\n+---+--------+---+------+------------+--------+\n| id|    name|age|salary|     address| nominee|\n+---+--------+---+------+------------+--------+\n|  1|  Manish| 26| 75000|       bihar|nominee1|\n|  2|  Nikita| 23|100000|uttarpradesh|nominee2|\n|  3|  Pritam| 22|150000|   Bangalore|   India|\n|  4|Prantosh| 17|200000|     Kolkata|   India|\n|  5|  Vikash| 31|300000|        null|nominee5|\n+---+--------+---+------+------------+--------+\n'

In [0]:
# DROPMALFORMED - 3 rec

employee_drop_malformed_df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .option("mode", "DROPMALFORMED")\
    .load("/FileStore/tables/employee_file.txt")

employee_drop_malformed_df.show()
"""
DROPMALFORMED ignored rows which had extra columns like id 3 and 4
+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        null|nominee5|
+---+------+---+------+------------+--------+
"""

+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        null|nominee5|
+---+------+---+------+------------+--------+



In [0]:
# FAILFAST - 0 rec
employee_failfast_df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .option("mode", "FAILFAST")\
    .load("/FileStore/tables/employee_file.txt")

employee_failfast_df.show()
"""
FAILFAST throws error even if 1 bad record is found. In our csv file we have id=3 and 4 as corrupted rows.

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 19) (ip-10-172-234-51.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/FileStore/tables/employee_file.txt.
"""


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2772480254097830>:7
      1 employee_failfast_df = spark.read.format("csv")\
      2     .option("header", "true")\
      3     .option("inferschema", "true")\
      4     .option("mode", "FAILFAST")\
      5     .load("/FileStore/tables/employee_file.txt")
----> 7 employee_failfast_df.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_c

In [0]:
# How to print bad records ?

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("id", IntegerType()),
    StructField("name", StringType()),
    StructField("age", IntegerType()),
    StructField("salary", IntegerType()),
    StructField("address", StringType()),
    StructField("nominee", StringType()),
    StructField("_corrupt_record", StringType())
])

# Default read mode is PERMISSIVE hence not passed here explicitly
employee_df = spark.read.format("csv")\
    .schema(schema)\
    .load("/FileStore/tables/employee_file.txt") 

employee_df.show(truncate=False)
"""
+----+--------+----+------+------------+--------+-------------------------------------------+
|id  |name    |age |salary|address     |nominee |_corrupt_record                            |
+----+--------+----+------+------------+--------+-------------------------------------------+
|null|name    |null|null  |address     |nominee |id,name,age,salary,address,nominee         |
|1   |Manish  |26  |75000 |bihar       |nominee1|null                                       |
|2   |Nikita  |23  |100000|uttarpradesh|nominee2|null                                       |
|3   |Pritam  |22  |150000|Bangalore   |India   |3,Pritam,22,150000,Bangalore,India,nominee3|
|4   |Prantosh|17  |200000|Kolkata     |India   |4,Prantosh,17,200000,Kolkata,India,nominee4|
|5   |Vikash  |31  |300000|null        |nominee5|null                                       |
+----+--------+----+------+------------+--------+-------------------------------------------+
"""



+----+--------+----+------+------------+--------+-------------------------------------------+
|id  |name    |age |salary|address     |nominee |_corrupt_record                            |
+----+--------+----+------+------------+--------+-------------------------------------------+
|null|name    |null|null  |address     |nominee |id,name,age,salary,address,nominee         |
|1   |Manish  |26  |75000 |bihar       |nominee1|null                                       |
|2   |Nikita  |23  |100000|uttarpradesh|nominee2|null                                       |
|3   |Pritam  |22  |150000|Bangalore   |India   |3,Pritam,22,150000,Bangalore,India,nominee3|
|4   |Prantosh|17  |200000|Kolkata     |India   |4,Prantosh,17,200000,Kolkata,India,nominee4|
|5   |Vikash  |31  |300000|null        |nominee5|null                                       |
+----+--------+----+------+------------+--------+-------------------------------------------+

Out[15]: '\n+----+--------+----+------+------------+-------

In [0]:
# How to store bad records ?

emp_schema = StructType([
    StructField("id", IntegerType()),
    StructField("name", StringType()),
    StructField("age", IntegerType()),
    StructField("salary", IntegerType()),
    StructField("address", StringType()),
    StructField("nominee", StringType()),
    StructField("_corrupt_record", StringType())
])

# Default read mode is PERMISSIVE hence not passed here explicitly
# Can write badRecords in new folder
employee_df = spark.read.format("csv")\
    .schema(emp_schema)\
    .option("badRecordsPath", "/FileStore/tables/badRecords")\
    .load("/FileStore/tables/employee_file.txt") 

employee_df.show(truncate=False)
"""
Since we passed extra col _corrupt_record for success records it printed as null
And corrupted records captured in /FileStore/tables/badRecords
+---+------+---+------+------------+--------+---------------+
|id |name  |age|salary|address     |nominee |_corrupt_record|
+---+------+---+------+------------+--------+---------------+
|1  |Manish|26 |75000 |bihar       |nominee1|null           |
|2  |Nikita|23 |100000|uttarpradesh|nominee2|null           |
|5  |Vikash|31 |300000|null        |nominee5|null           |
+---+------+---+------+------------+--------+---------------+
"""



+---+------+---+------+------------+--------+---------------+
|id |name  |age|salary|address     |nominee |_corrupt_record|
+---+------+---+------+------------+--------+---------------+
|1  |Manish|26 |75000 |bihar       |nominee1|null           |
|2  |Nikita|23 |100000|uttarpradesh|nominee2|null           |
|5  |Vikash|31 |300000|null        |nominee5|null           |
+---+------+---+------+------------+--------+---------------+



In [0]:
%fs 
ls dbfs:/FileStore/tables/badRecords/20250406T131422/bad_records/part-00000-637874de-b956-47e9-b1b6-444fec68bf3a

path,name,size,modificationTime
dbfs:/FileStore/tables/badRecords/20250406T131422/bad_records/part-00000-637874de-b956-47e9-b1b6-444fec68bf3a,part-00000-637874de-b956-47e9-b1b6-444fec68bf3a,661,1743945264000


In [0]:
"""
%fs
ls /FileStore/tables/badRecords/ <--- Keep doing ls to reach to part json file
"""

bad_data_df = spark.read.format("json").load("/FileStore/tables/badRecords/20250406T131422/bad_records/part-00000-637874de-b956-47e9-b1b6-444fec68bf3a")

bad_data_df.show(truncate=False)
"""
|path                                    |reason                                                                                                                          |record                                     |
+----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+
|dbfs:/FileStore/tables/employee_file.txt|java.lang.NumberFormatException: For input string: "id"                                                                         |id,name,age,salary,address,nominee         |
|dbfs:/FileStore/tables/employee_file.txt|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 3,Pritam,22,150000,Bangalore,India,nominee3|3,Pritam,22,150000,Bangalore,India,nominee3|
|dbfs:/FileStore/tables/employee_file.txt|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 4,Prantosh,17,200000,Kolkata,India,nominee4|4,Prantosh,17,200000,Kolkata,India,nominee4|
"""

+----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+
|path                                    |reason                                                                                                                          |record                                     |
+----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+
|dbfs:/FileStore/tables/employee_file.txt|java.lang.NumberFormatException: For input string: "id"                                                                         |id,name,age,salary,address,nominee         |
|dbfs:/FileStore/tables/employee_file.txt|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 3,Pritam,